# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import r2_score

In [2]:
pth1 = '/datasets/insurance.csv'
pth2 = '/folder_2/data.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [3]:
print('shape =', df.shape)
print('nan =', df.isna().sum().sum())
print('dupl =', df.duplicated().sum())
print()
print(df.info())

df.head()

shape = (5000, 5)
nan = 0
dupl = 153

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


**153 Дубликата. Остальное пока в порядке**

### Предобработка данных.

In [4]:
df[['Возраст', 'Зарплата']] = df[['Возраст', 'Зарплата']].astype(int)
print()
df.drop_duplicates(inplace=True)
print('dupl =', df.duplicated().sum())
print()

df.info()


dupl = 0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


In [5]:
df.columns = map(str.lower, df.columns)
df.columns = (df.columns.str.replace(' ', '_', regex=True))
print(df.columns)

Index(['пол', 'возраст', 'зарплата', 'члены_семьи', 'страховые_выплаты'], dtype='object')


**Колонки в питоний вид, дубли решил вынести, я не уверен, но думаю что подобные дубликаты могут негативно сказаться на обучении модели.**

## Умножение матриц

In [6]:
X = []
P = []
for i in range(3):
    row = []
    row2 = []
    for j in range(3):
        row.append(np.random.normal())
        row2.append(np.random.normal())
    X.append(row)
    P.append(row2)
    
X = np.array(X)
P = np.array(P)
np.linalg.inv(X)
np.linalg.inv(P)

y = []
for i in range(3):
    y.append(X[i][2])
    X[i][2] = 1
    
E = np.eye(3)
# Создаю единичную матрицу, две рандомных матрицы, у первой изымаю столбец признаков, заменяю их столбцом с единицами.
print(E) 
print(X)
print(y)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 0.68183124  1.53436836  1.        ]
 [ 0.65534382 -1.15896024  1.        ]
 [-0.21163314 -0.80787481  1.        ]]
[-0.4333805697335855, 1.6208404420601161, -2.357305244087853]


In [7]:
# Раскрываем скобки с транспонированием по правилу (𝐴𝐵)𝑇=𝐵𝑇𝐴𝑇

w1 = np.linalg.inv(P.T @ X.T @ X @ P) @ P.T @ X.T @ y

# Далее выделим квадратные матрицы, поскольку только у квадратных могут быть обратные.
# (𝑋.𝑇@𝑋) квадратная матрица,
# 𝑃 и 𝑃.𝑇 по определению квадратные обратимые матрицы

w1 = np.linalg.inv(P.T @ X.T @ X @ P) @ P.T @ X.T @ y

# Раскроем скобки по правилу (𝐴𝐵)−1=𝐵−1𝐴−1

w1 = np.linalg.inv(P) @ np.linalg.inv(X.T @ X) @ np.linalg.inv(P.T) @ P.T @ X.T @ y

# Поскольку 𝐴−1𝐴=𝐸, а при умножении на единичную матрицу выражение не изменяется (𝐴𝐸=𝐸𝐴=𝐴), то

w1 = np.linalg.inv(P) @ np.linalg.inv(X.T @ X) @ E @ X.T @ y
w1 = np.linalg.inv(P) @ np.linalg.inv(X.T @ X) @ X.T @ y

# Так как  𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦, то получаем связь весов

w = np.linalg.inv(X.T @ X) @ X.T @ y

w1 = np.linalg.inv(P) @ w

# Подставляем получившееся выражение 𝑤1=𝑃−1𝑤 в фрмулу для предсказаний  𝑎1=𝑋𝑃𝑤1

a1 = X @ P @ np.linalg.inv(P) @ w

In [8]:
a = X @ np.array(w)

In [9]:
(np.round(a, 3) == np.round(a1, 3)).sum()

3

**a равно а1.**

## Изменится ли r2_score.

In [10]:
features = df.drop('страховые_выплаты', axis=1)
target = df['страховые_выплаты']

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010046633359


**Вычисляю исходный r2_score**

In [11]:
matrix = []
for i in range(4):
    row = []
    for j in range(4):
        row.append(np.random.normal())
    matrix.append(row)
matrix = np.array(matrix)
print(matrix)

[[ 0.76610447  0.79149042  0.30868947  0.86341807]
 [ 0.80950502  1.44648183 -1.1899352   0.57094454]
 [-1.03018502  1.86672723 -0.93675535 -0.67259308]
 [ 0.8539778   1.55144047 -1.70310226  0.07521149]]


In [12]:
np.linalg.inv(matrix)

array([[ 1.07843635, -2.27561946, -0.32750709,  1.96555177],
       [ 0.8603484 , -0.90494511,  0.39981077,  0.56830508],
       [ 1.27836581, -1.81101548,  0.19357868,  0.80340053],
       [-1.04441931,  3.49617314, -0.14511793, -2.55221358]])

**Создаю матрицу рандомом подсказанным в теории, когда пытался указать цифры в цикле, даже через коунтер который немного изменял их, постоянно получалась необратимая матрица)) Пришлось рандом этот искать))**

**Матрица обратимая, едем дальше.**

In [13]:
m_features = features @ matrix # Умножаю датафрейм на обратимую матрицу.

In [14]:
model = LinearRegression()
model.fit(m_features, target)
predictions = model.predict(m_features)
print(r2_score(target, predictions))

0.4302010046631788


**Ответ: r2_score не изменился..**

**Обоснование: Я если честно в недоумении))**

## Алгоритм преобразования

**Алгоритм**

Мы возьмём матрицу которая была результатом умножения нашего датафрейма на обратимую матрицу(m_features).
Далее инвертируем нашу обратимую матрицу(matrix) и умножим m_features на inv(matrix).

m_features = features @ matrix

features = m_features @ inv(matrix)

In [15]:
features_test = m_features @ np.linalg.inv(matrix)
features_test = np.round(features_test)
features_test = features_test.astype(int)

In [16]:
features_test.head()

,0,1,2,3
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


**Обоснование**

Не меняется потому-что мы возвращаем данные в исходный вид.

## Проверка алгоритма

### r2_score модели до преобразований.

In [17]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010046633359


### r2_score модели после умножения на обратимую матрицу.

In [18]:
model = LinearRegression()
model.fit(m_features, target)
predictions = model.predict(m_features)
print(r2_score(target, predictions))

0.4302010046631788


### r2_score модели после обратного преобразования.

In [19]:
model = LinearRegression()
model.fit(features_test, target)
predictions = model.predict(features_test)
print(r2_score(target, predictions))

0.4302010046633359


**Вывод**

**Данные можно зашифровать рандомной обратимой матрицей и вернуть их в исходное положение, во время преобразований выводы линейной регрессии не изменяются.**

**Также думаю стоит добавить что методом перебора такую матрицу-ключ подобрать очень проблематично и данные таким образом обладают некой степенью защиты.**